In [9]:
import sqlite3
import pandas as pd
import bcrypt

In [10]:
conn = sqlite3.connect('todo_test.db')
cursor = conn.cursor()

In [11]:
cursor.execute('''CREATE TABLE IF NOT EXISTS todos (
    todo_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER,
    title TEXT NOT NULL,
    category TEXT,
    priority INTEGER CHECK (priority BETWEEN 1 AND 5),
    description TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    due_date DATE,
    done BOOLEAN DEFAULT 0,
    FOREIGN KEY (user_id) REFERENCES users(user_id) ON DELETE CASCADE
);''')

In [13]:
cursor.executemany('''
INSERT INTO todos (user_id, title, category, priority, description, due_date, done)
VALUES (?, ?, ?, ?, ?, ?, ?);
''', [
    (1, 'SQL lernen', 'Lernen', 4, 'Übung mit JOINs und Views', '2025-11-10', 0),
    (1, 'Projektbesprechung', 'Team', 2, 'Git Branch Strategie diskutieren', '2025-11-03', 1),
    (2, 'User-Login fertigstellen', 'Programmierung', 5, 'bcrypt und Sperrlogik testen', '2025-11-05', 0),
    (2, 'Dokumentation schreiben', 'Doku', 3, 'Screenshots und ER-Diagramm einfügen', '2025-11-09', 0),
    (3, 'GUI-Design', 'Frontend', 4, 'PyQt Layout finalisieren', '2025-11-07', 1),
    (3, 'Bugfix in SQL Query', 'Backend', 3, 'ORDER BY Fälligkeitsdatum', '2025-11-04', 0),
    (4, 'Code-Review', 'Team', 2, 'Anna’s Merge Request prüfen', '2025-11-02', 1),
    (4, 'Testing', 'QA', 5, 'Unit Tests für user_login', '2025-11-06', 0),
    (5, 'Testbenutzer löschen', 'Admin', 1, 'Cleanup nach Demo', '2025-11-08', 0)
])

conn.commit()

In [6]:
cursor.execute('''DROP TABLE todos''')

In [14]:
pd.read_sql('SELECT * FROM todos', conn)


,todo_id,user_id,title,category,priority,description,created_at,due_date,done
0,1,1,SQL lernen,Lernen,4,Übung mit JOINs und Views,2025-11-04 12:34:52,2025-11-10,0
1,2,1,Projektbesprechung,Team,2,Git Branch Strategie diskutieren,2025-11-04 12:34:52,2025-11-03,1
2,3,2,User-Login fertigstellen,Programmierung,5,bcrypt und Sperrlogik testen,2025-11-04 12:34:52,2025-11-05,0
3,4,2,Dokumentation schreiben,Doku,3,Screenshots und ER-Diagramm einfügen,2025-11-04 12:34:52,2025-11-09,0
4,5,3,GUI-Design,Frontend,4,PyQt Layout finalisieren,2025-11-04 12:34:52,2025-11-07,1
5,6,3,Bugfix in SQL Query,Backend,3,ORDER BY Fälligkeitsdatum,2025-11-04 12:34:52,2025-11-04,0
6,7,4,Code-Review,Team,2,Anna’s Merge Request prüfen,2025-11-04 12:34:52,2025-11-02,1
7,8,4,Testing,QA,5,Unit Tests für user_login,2025-11-04 12:34:52,2025-11-06,0
8,9,5,Testbenutzer löschen,Admin,1,Cleanup nach Demo,2025-11-04 12:34:52,2025-11-08,0


Create Table users

In [17]:
cursor.execute('''CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT,
    last_name TEXT,
    alias TEXT,
    password TEXT NOT NULL,
    failed_attempts INT DEFAULT 0,
    freezed BOOLEAN DEFAULT FALSE,
    is_admin BOOLEAN DEFAULT FALSE,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);''')

In [12]:
cursor.execute('''DROP TABLE users''')

REGISTER

In [35]:
def register_user(first_name, last_name, alias, plain_password, is_admin=False):
    cursor.execute("SELECT alias FROM users WHERE alias=?", (alias,))
    if cursor.fetchone():
        print(f'{alias} exists already!')
        return
    hashed_pw = bcrypt.hashpw(plain_password.encode('utf-8'), bcrypt.gensalt()).decode('utf-8')
    cursor.execute("""
        INSERT INTO users (first_name, last_name, alias, password, is_admin)
        VALUES (?, ?, ?, ?, ?)
    """, (first_name, last_name, alias, hashed_pw, is_admin))
    conn.commit()
    print(f'{alias} has been registered successfully!')

In [20]:
register_user('Mengjia','Liu','mliu','test123',True)
register_user('Anne','Noe','Anoe','test123',True)
register_user('Nicole','Barsch','nbarsch','test123',True)
register_user('Valerii','Yantselovski','vyantselovski','test123',True)
register_user('Valentin','','vl','test123',False)

mliu has been registered successfully!
Anoe has been registered successfully!
nbarsch has been registered successfully!
vyantselovski has been registered successfully!
vl has been registered successfully!


In [36]:
pd.read_sql('SELECT * FROM users', conn)

,user_id,first_name,last_name,alias,password,failed_attempts,freezed,is_admin,created_at
0,1,Mengjia,Liu,mliu,$2b$12$WsqVn0vcVwIG3x3QfTNUWO7Ef4G4n/PwtPwPr8....,0,0,1,2025-11-04 12:45:33
1,2,Anne,Noe,Anoe,$2b$12$ujEHlHKuIK0fPdfpUb02oOf.Dtp2xJ4kTEGPmDf...,0,0,1,2025-11-04 12:45:33
2,3,Nicole,Barsch,nbarsch,$2b$12$K4mYU10KVNpG.oikN4YEt.PnUDDsaZIvGxcZxBw...,0,0,1,2025-11-04 12:45:33
3,4,Valerii,Yantselovski,vyantselovski,$2b$12$9M69VdVc3uXPeQmaQkUDO.j2/rgUyzurhClYx.6...,0,0,1,2025-11-04 12:45:33
4,5,Valentin,,vl,$2b$12$AJ6TlwVlPpJ2gpNDNMm7deHLuRw5K1piWuR5ikx...,0,0,0,2025-11-04 12:45:33


Welcome

LOGIN

In [41]:
def login(alias, plain_password):
    cursor.execute("""
        SELECT user_id, password, failed_attempts, freezed, is_admin
        FROM users WHERE alias=?
    """, (alias,))
    user = cursor.fetchone()

    if not user:
        print(f'{alias} does not exist!')
        return False

    user_id, stored_hash, failed_attempts, freezed, is_admin = user

    if freezed:
        print('This account is blocked! Please contact admin!')
        return False

    if bcrypt.checkpw(plain_password.encode('utf-8'), stored_hash.encode('utf-8')):
        cursor.execute("UPDATE users SET failed_attempts=0 WHERE user_id=?", (user_id,))
        conn.commit()
        print(f'Welcome back, {alias}!')

        return {
            "user_id": user_id,
            "alias": alias,
            "is_admin": bool(is_admin),
        }
    else:
        failed_attempts += 1
        if failed_attempts >= 3:
            cursor.execute("UPDATE users SET freezed=1, failed_attempts=? WHERE user_id=?", (failed_attempts, user_id))
            print(f'You have given incorrect password for three times. The account of {alias} has been blocked!')
        else:
            cursor.execute("UPDATE users SET failed_attempts=? WHERE user_id=?", (failed_attempts, user_id))
            print(f'Wrong password! You have (3 - {failed_attempts}) times left!')
        conn.commit()
        return False


In [42]:
user = login('vl','test123')
user_main(user)

Welcome back, vl!

 Welcome back, vl!

 Main menu
1 - personal information

2️ - my todos

0️ - logout


 Your information:
user_id: 5
alias: vl
is_admin: False

Functions:
1 - change my information

2 - delete my account

0 - exit


Change my information:

1 - change my name

2 - change my password

0 - exit


KeyboardInterrupt: Interrupted by user

In [26]:
def main_menu(user):
    if not user:
        print('Login Failed!')
        return

    print(f'\n Welcome back, {user['alias']}!')

    while True:
        print('\n Main menu')
        print('1 - personal information\n')
        print('2️ - my todos\n')
        if user['is_admin']:
            print('️3 - user management\n')
        print('0️ - logout\n')

        choice = input('Please choose your option:').strip()

        if choice == "1":
            show_profile(user)
        elif choice == "2":
            show_todos(user)
        elif choice == "3" and user["is_admin"]:
            user_management()
        elif choice == "0":
            print('Logout successful! See you next time!')
            break
        else:
            print('Invalid input!')

In [27]:
def show_profile(user):
    print("\n Your information:")
    for k, v in user.items():
        print(f"{k}: {v}")
    while True:
        print('\nFunctions:')
        print('1 - change my information\n')
        print('2 - delete my account\n')
        print('0 - exit\n')
        choice = input('Please choose your option:').strip()

        if choice == "1":
            change_information(user)
        elif choice == "2":
            delete_account()
            break
        elif choice == "0":
            break
        else:
            print('Invalid input!')


In [28]:
def change_information(user):
    print('\nChange my information:')
    print('\n1 - change my name')
    print('\n2 - change my password')
    print('\n0 - exit')
    choice = input('Please choose your option:').strip()

    if choice == '1':
        new_first = input('Your first name:').strip()
        new_last = input('Your last name').strip()

        if new_first or new_last:
            cursor.execute("""
                UPDATE users
                SET first_name = COALESCE(NULLIF(?, ''), first_name),
                    last_name  = COALESCE(NULLIF(?, ''), last_name)
                WHERE user_id = ?
            """, (new_first, new_last, user["user_id"]))
            conn.commit()
            print('Your information has been updated successfully!')
        else:
            print('No changes were taken!')


    elif choice == '2':
        new_pw = input('Your new password:').strip()
        confirm_pw = input('Your new passord again:').strip()
        if new_pw != confirm_pw:
            print('Passwords are not identical!')
            return

        hashed_pw = bcrypt.hashpw(new_pw.encode('utf-8'), bcrypt.gensalt()).decode('utf-8')
        cursor.execute("UPDATE users SET password=? WHERE user_id=?", (hashed_pw, user["user_id"]))
        conn.commit()
        print('Your password has been updated successfully!')

    elif choice == '0':
        return

    else:
        print('Invalid input!')



In [29]:
def delete_account(user):
    alias = user["alias"]
    confirmation = input(f'\nType: "{alias} confirms to delete." to confirm.\n> ")

    if confirmation != f"{alias} confirms to delete.":
        print('Pattern does not match! Deletion was cancelled!')
        return False

    cursor.execute("DELETE FROM users WHERE user_id=?", (user["user_id"],))
    conn.commit()

    print(f'{alias}\'s account has been deleted!')' )
    return True


NameError: name 'user' is not defined

In [ ]:
def show_todos(user):
    cursor.execute("SELECT * FROM todos WHERE user_id=?", (user["user_id"],))
    rows = cursor.fetchall()

    if not rows:
        print("You have no tasks currently!")
    else:
        print("\nYour tasks:")
        for row in rows:
            print(row)


In [ ]:
def user_management(user):
    while True:
        print('\nUser management:')
        print('\n1 - show all users')
        print('\n2 - reset password')
        print('\n3 -  freeze / defreeze user')
        print('\n0 - exit\n')

        choice = input('Please choose your option:').strip()

        if choice == "1":
            show_users()
        elif choice == "2":
            reset_password()
        elif choice == "3":
            account_freeze()
        elif choice == "0":
            break
        else:
            print('Invalid input!')


In [ ]:
def show_users():
    cursor.execute("""
        SELECT user_id, first_name, last_name, alias, is_admin, freezed, failed_attempts, created_at
        FROM users ORDER BY user_id
    """)
    rows = cursor.fetchall()
    for row in rows:
        print(row)


In [ ]:
def reset_password():
    try:
        target_id = int(input('Please enter the user id:').strip())
    except ValueError:
        print('Invalid input!')
        return

    new_pw = input('Please enter new password').strip()
    if not new_pw:
        print('Password is empty!')
        return

    confirm_pw = input('Your new passord again:').strip()
    if new_pw != confirm_pw:
        print('Passwords are not identical!')
        return


    hashed_pw = bcrypt.hashpw(new_pw.encode('utf-8'), bcrypt.gensalt()).decode('utf-8')
    cursor.execute("UPDATE users SET password=?, failed_attempts=0, freezed=0 WHERE user_id=? AND is_admin=0",
                   (hashed_pw, target_id))
    if cursor.rowcount:
        conn.commit()
        print(f' The password of User {target_id} has been updated!')
    else:
        print('You can not reset password of ohter administrator!')


In [ ]:
def account_freeze():
    try:
        target_id = int(input('Please enter the user id:').strip())
    except ValueError:
        print('Invalid input!')
        return

    cursor.execute("SELECT alias, freezed FROM users WHERE user_id=?", (target_id,))
    row = cursor.fetchone()
    if not row:
        print('Id does not exist!')
        return

    if freezed:
        new_state = 0
        action = 'defreezed'
    else:
        new_state = 1
        action = 'freezed'

    cursor.execute("UPDATE users SET freezed=?, failed_attempts=0 WHERE user_id=?", (new_state, target_id))
    conn.commit()
    print(f' User {alias} is now {action}')
